`hybrid-vertex.spotify_train_3.unique_track_features`

+

`hybrid-vertex.spotify_train_3.unique_artist_features`

+

`hybrid-vertex.spotify_mpd.playlists_nested`

Trainv3-create-enriched-data


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.enriched_data` AS (
    SELECT
    a.* except(tracks),
      ARRAY(
    SELECT
      AS STRUCT CAST(track.pos AS int64) AS pos_seed,
      track.artist_name AS artist_name_seed,
      track.track_uri AS track_uri_seed,
      track.track_name AS track_name_seed,
      track.album_uri AS album_uri_seed,
      track.artist_uri AS artist_uri_seed,
      CAST(track.duration_ms AS float64) AS duration_ms_seed,
      track.album_name AS album_name_seed,
      tf.tracks_playlist_titles AS tracks_playlist_titles,
      tf.track_pop AS track_pop,
      af.artist_pop AS artist_pop,
      af.artist_genres AS artist_genres,
      af.artist_followers AS artist_followers
    FROM
      UNNEST(tracks) as track
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_track_features` AS tf --track features
    ON
      (track.track_uri = tf.track_uri)
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_artist_features` AS af --artist features
      ON
      (track.artist_uri = af.artist_uri)
      ) AS tracks
  FROM 
  `hybrid-vertex.spotify_mpd.playlists_nested` as a)


cross_join_songxplaylist_struct_query

`hybrid-vertex.spotify_train_3.ordered_position_training`



In [ ]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.ordered_position_training` AS (
  WITH
    -- get every combination of song and its parent playlist
    unnest_cross AS(
    SELECT
      b.*,
      CONCAT(b.pid,"-",track.pos_seed) AS pid_pos_id,
      CAST(track.pos_seed AS int64) AS pos,
      track.artist_name_seed,
      track.track_uri_seed,
      track.artist_uri_seed,
      track.track_name_seed,
      track.album_uri_seed,
      CAST(track.duration_ms_seed AS float64) AS duration_ms_seed,
      track.album_name_seed,
      tracks_playlist_titles AS tracks_playlist_titles,
      track_pop AS track_pop_seed,
      artist_pop AS artist_pop_seed,
      artist_genres AS artist_genres_seed,
      artist_followers AS artist_followers_seed
    FROM (
      SELECT
        * EXCEPT(duration_ms),
        duration_ms AS duration_ms_playlist
      FROM
        `hybrid-vertex.spotify_train_3.enriched_data`) AS b
    CROSS JOIN
      UNNEST(tracks) AS track)
  SELECT
    a.* EXCEPT(tracks,
      num_tracks,
      num_artists,
      num_albums,
      num_followers,
      num_edits),
    ARRAY(
    SELECT
      AS STRUCT CAST(track.pos_seed AS int64) AS pos_seed,
      track.artist_name_seed AS artist_name_seed,
      track.track_uri_seed AS track_uri_seed,
      track.track_name_seed AS track_name_seed,
      track.album_uri_seed AS album_uri_seed,
      track.artist_uri_seed AS artist_uri_seed,
      CAST(track.duration_ms_seed AS float64) AS duration_ms_seed,
      track.album_name_seed AS album_name_seed,
      tracks_playlist_titles AS tracks_playlist_titles,
      track_pop AS track_pop_seed,
      artist_pop AS artist_pop_seed,
      artist_genres AS artist_genres_seed,
      artist_followers AS artist_followers_seed
    FROM
      UNNEST(tracks) AS track
    WHERE
      CAST(track.pos_seed AS int64) < a.pos ) AS seed_playlist_tracks,
    ----- seed track part
    trx.pos_seed AS pos_seed_track,
    trx.artist_name_seed AS artist_name_seed_track,
    trx.artist_uri_seed AS artist_uri_seed_track,
    trx.track_name_seed AS track_name_seed_track,
    trx.track_uri_seed AS track_uri_seed_track,
    trx.album_name_seed AS album_name_seed_track,
    trx.album_uri_seed AS album_uri_seed_track,
    trx.duration_ms_seed AS duration_seed_track
  FROM
    unnest_cross AS a -- with statement
    ,
    UNNEST(a.tracks) AS trx
  WHERE
    CAST(trx.pos_seed AS int64) = a.pos-1 );
  -- LIMIT
  -- 100;

Trainv3-clean-track-features

`hybrid-vertex.spotify_train_3.train`


In [ ]:
%%bigquery
create or replace table `hybrid-vertex.spotify_train_3.train` as (
WITH
  playlist_features_clean AS (
  SELECT
    pid_pos_id as pid_pos_id,
    SUM(trx.duration_ms_seed) AS duration_ms_seed_pl,
    COUNT(1) AS n_songs,
    COUNT(DISTINCT trx.artist_name_seed) AS num_artists,
    COUNT(DISTINCT trx.album_uri_seed) AS num_albums,
  FROM
    `hybrid-vertex.spotify_train_3.ordered_position_training`,
    UNNEST(seed_playlist_tracks) AS trx
  GROUP BY
    pid_pos_id)
SELECT
  a.*,
  b.* except(pid_pos_id)
FROM
  `hybrid-vertex.spotify_train_3.ordered_position_training` a
INNER JOIN
  playlist_features_clean b
ON
  a.pid_pos_id = b.pid_pos_id)